# <center>ML-Recommendation System</center>

Dataset acquired from GroupLens

In [ ]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Now let's read each file into their Dataframes:

In [ ]:
movies_df = pd.read_csv('data/movies.csv')
movies_df.head()

In [ ]:
ratings_df = pd.read_csv('data/ratings.csv')
ratings_df.head()

Let's also remove the year from the __title__ column by using pandas' replace function and store in a new __year__ column.

In [ ]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

Let's also split the values in the __Genres__ column into a __list of Genres__ to simplify future use. This can be achieved by applying Python's split string function on the correct column.

In [ ]:
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

Since keeping genres in a list format isn't optimal for the content-based recommendation system technique, we will use the One Hot Encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature. This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable since genres won't be important for our first recommendation system.

In [ ]:
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

Next, let's look at the ratings dataframe.

In [ ]:
ratings_df.head()

Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it.

In [ ]:
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

Let's create user input to recommend movies to:

Write movie names with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .

In [ ]:
userInput = [
            {'title':'Return of the King, The', 'rating':5},
            {'title':'Fellowship of the Ring, The', 'rating':5},
            {'title':'Inception', 'rating':4},
            {'title':"Two Towers, The", 'rating':5},
            {'title':'Interstellar', 'rating':5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

With the input complete, let's extract the input movie's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movie's title and then merging this subset with the input dataframe. We also drop unnecessary columns

In [ ]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#merging it so we can get the movieId
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)

inputMovies

In [ ]:
inputId

We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.

In [ ]:
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.

In [ ]:
userMovies = userMovies.reset_index(drop=True)

userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

Let's turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column.

In [ ]:
inputMovies['rating']

In [ ]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
userProfile

Now, we have the weights for every of the user's preferences. Using this, we can recommend movies that satisfy the user's preferences.

Let's start by extracting the genre table from the original dataframe:

In [ ]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

In [ ]:
genreTable.shape

With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.

In [ ]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

In [ ]:
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
recommendationTable_df.head()

Now here's the recommendation table!

In [ ]:
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

---------

### User Input

In [ ]:
movieCount = int(input("Enter number of movies: "))

In [ ]:
userInputMovie = []
userInputRating = []
x = range(movieCount)
for n in x:
    userInputMovie_1 = input("Please enter a movie name: ").capitalize()
    userInputRating_1 = float(input("Please enter rating for that movie out of 5: "))
    userInputMovie.append(userInputMovie_1)
    userInputRating.append(userInputRating_1)
movieData = {"title":userInputMovie, "rating":userInputRating}
#inputTable = pd.DataFrame([[userInputMovie, userInputRating]], columns = ["title", "rating"])
inputTable = pd.DataFrame(movieData)
inputTable

In [ ]:
#Filtering out the movies by title
inputId_1 = movies_df[movies_df['title'].isin(inputTable['title'].tolist())]
#merging it so we can get the movieId
inputMovies_1 = pd.merge(inputId_1, inputTable)
#Dropping information we won't use from the input dataframe
inputMovies_1 = inputMovies_1.drop('genres', 1).drop('year', 1)

inputMovies_1

In [ ]:
userMovies_1 = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies_1['movieId'].tolist())]
userMovies_1

In [ ]:
userMovies_1 = userMovies_1.reset_index(drop=True)

userGenreTable_1 = userMovies_1.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable_1

In [ ]:
inputMovies_1['rating']

In [ ]:
#Dot produt to get weights
userProfile_1 = userGenreTable_1.transpose().dot(inputMovies_1['rating'])
userProfile_1

In [ ]:
#Now let's get the genres of every movie in our original dataframe
genreTable_1 = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable_1 = genreTable_1.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable_1.head()

In [ ]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df_1 = ((genreTable_1*userProfile_1).sum(axis=1))/(userProfile_1.sum())
recommendationTable_df_1.head()

In [ ]:
recommendationTable_df_1 = recommendationTable_df_1.sort_values(ascending=False)
recommendationTable_df_1.head()

In [ ]:
recomendedMovies = movies_df.loc[movies_df['movieId'].isin(recommendationTable_df_1.head(20).keys())]
recomendedMovies

In [ ]:
movieNameSearch = input("Please enter the name of the movie: ").capitalize()
boolean_finding = movies_df[movies_df["title"].str.contains(movieNameSearch)] 
print(boolean_finding)

movieNameSearch = input("Please enter the name of the movie: ")
boolean_finding = movies_df['title'].str.contains(movieNameSearch).any()
if(boolean_finding):
    print("Yes, the movie is present in the database")
else:
    print("No, the movie is NOT present in the database")

In [ ]:
import pymongo
from config import mongoDBinfo
from pymongo import MongoClient
client = pymongo.MongoClient(mongoDBinfo)
db = client.test

In [ ]:
print(client.list_database_names())

https://www.kaggle.com/ashishsaxena2209/tutorial-to-use-mongodb-atlas-cloud-service

In [ ]:
mydb = client['mydatabase']
mycol = mydb['movies']

In [ ]:
collist = mydb.list_collection_names()
if "movies" in collist:
    print("The collection exists.")
else:
    print("Couldn't find the collection.")

In [ ]:
recomendedMoviesDict = recomendedMovies.to_dict()

In [ ]:
recomendedMoviesDict = [recomendedMoviesDict]

x = mycol.insert_many(recomendedMoviesDict)

#print list of the _id values of the inserted documents:
print(x.inserted_ids)